In [26]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv('insurance.csv')

In [22]:
X, y = df.drop('charges', axis=1), df['charges']
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


In [4]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## 1. Removing features with low variance 
(if lower than threshold)

In [36]:
# Encodes target labels with value between 0 and n_classes-1
le = LabelEncoder()
X_enc = X.copy()

for col in ['age', 'sex', 'region', 'smoker']:
    X_enc[col] = le.fit_transform(X_enc[col])

In [43]:
from sklearn.feature_selection import VarianceThreshold

# Default threshold = 0 -> same value in every sample of that feature
selector = VarianceThreshold(threshold=0.2)

In [52]:
data_selected = selector.fit_transform(X_enc)
X_sel = pd.DataFrame(data_selected, columns=X_enc.columns[selector.get_support()])

In [54]:
# Output: removed smoker
X_sel.head()

,age,sex,bmi,children,region
0,1.0,0.0,27.900,0.0,3.0
1,0.0,1.0,33.770,1.0,2.0
2,10.0,1.0,33.000,3.0,2.0
3,15.0,1.0,22.705,0.0,1.0
4,14.0,1.0,28.880,0.0,1.0


## 2. Univariate feature selection